In [ ]:
import numpy as np; import matplotlib.pyplot as plt

def rate_cm_to_rate_ps(rate_cm): # stolen from chris' python code
    return((200.0E-12*pi*c_vac*rate_cm))

def overlap(direc, chl_index, car_index, plot):
    car_file = "{}/car_{:02d}.dat".format(direc, car_index)
    chl_file = "{}/chl_{:02d}.dat".format(direc, chl_index)
    car = np.loadtxt(car_file)
    chl = np.loadtxt(chl_file)
    # uncomment the following three lines to check that the integrals are properly normalised (wrt the set of wavenumbers we generated above!)
    #print("integral of car lineshape = {:10.6e}".format(np.trapz(car, x=wn)))
    #print("integral of chl a(w) lineshape = {:10.6e}".format(np.trapz(chl[:, 0], x=wn)))
    #print("integral of chl f(w) lineshape = {:10.6e}".format(np.trapz(chl[:, 1], x=wn)))
    # chlorophyll lineshapes in these files are saved as a(\omega), f(\omega)
    car_to_chl = car * chl[:, 0]
    chl_to_car = car * chl[:, 1]
    car_to_chl_overlap = np.trapz(car_to_chl, x=wn)
    chl_to_car_overlap = np.trapz(chl_to_car, x=wn)
    print("{:2d} {:2d} {:10.6e} {:10.6e}".format(chl_index, car_index, chl_to_car_overlap, car_to_chl_overlap))
    # plot stuff
    if (plot):
        fig, ax = plt.subplots(2, sharex=True)
        ax[0].plot(wn, car, label=r'car')
        ax[0].plot(wn, chl[:, 0], label=r'chl $A(\omega)$')
        ax[0].plot(wn, chl[:, 1], label=r'chl $F(\omega)$')
        ax[1].plot(wn, car_to_chl, label=r'car $ \rightarrow $ chl, overlap = {:10.6f}'.format(car_to_chl_overlap))
        ax[1].plot(wn, chl_to_car, label=r'chl $ \rightarrow $ car, overlap = {:10.6f}'.format(chl_to_car_overlap))
        ax[0].legend(); ax[1].legend()
        ax[1].set_xlabel("Wavenumber $ (cm^{-1}) $")
        ax[0].set_xlim([10000,20000])
        plt.savefig("{}/car_{:02d}_chl_{:02d}.pdf".format(direc, car_index, chl_index))
        plt.show()
        plt.close(fig)

    # we append this to an array to check against the overlaps i calculate in the C++
    return [chl_index, car_index, chl_to_car_overlap, car_to_chl_overlap]


direc = "/home/callum/code/spectra/out/NLLZ/7_PROD_2/2/1000"
cvac = 299792458.0
ns = 2048
n_chl = 14
eig = np.loadtxt("{}/eigvecs.out".format(direc))
Jij = np.loadtxt("{}/J_ij.out".format(direc))
with np.printoptions(precision=3, suppress=True, linewidth=np.nan):
    print(eig)
    print(Jij)

# this is the set of wavenumbers we generate spectra for - we need them to integrate over in the trapz calls
wn = np.array([i / (ns * cvac * 100 * 1E-15) for i in range(ns)])

In [ ]:
# note that FC overlaps aren't included here
print("Calculated exciton-carotenoid couplings:")
for chl_index in range(n_chl):
    for lut in range(n_chl,n_chl + 2): # lut 1 and 2 are the final two rows/columns in the eigenvector/coupling matrices
        ji = 0.0
        for m in range(n_chl):
           #ji += eig[m, chl_index] * eig[n, chl_index] * Jij[m, lut] * Jij[n, lut]
           ji += eig[m, chl_index] * Jij[m, lut]
           #print(eig[m, chl_index], eig[n, chl_index], Jij[m, lut], Jij[n, lut], eig[m, chl_index] * eig[n, chl_index] * Jij[m, lut] * Jij[n, lut], ji)
            
        if (lut == 14):
            print("{:2d} {} {:10.6e}".format(chl_index, "620", ji))
        else:
            print("{:2d} {} {:10.6e}".format(chl_index, "621", ji))

            
overlaps = []
print("\nExciton-vibronic overlaps:")
for chl_index in range(n_chl):
    for car_index in range(16):
        overlaps.append(overlap(direc, chl_index, car_index, False))

In [ ]:
# the numbering of the carotenoid states is different in the C++ code, so just take the columns with the overlaps in and compare those
py_overlaps = np.transpose(np.array(overlaps))
c_overlaps  = np.transpose(np.array(np.loadtxt("../out/overlaps.dat")))
print(np.allclose(py_overlaps[2], c_overlaps[2]))
print(np.allclose(py_overlaps[3], c_overlaps[3]))